In [ ]:
# default_exp datasets.steam

# Steam Dataset
> Implementation of Steam games dataset.

Steam is the world's most popular PC Gaming hub, with over 6,000 games and a community of millions of gamers. With a massive collection that includes everything from AAA blockbusters to small indie titles, great discovery tools are a highly valuable asset for Steam.

Steam Video Game and Bundle Data

These datasets contain reviews from the Steam video game platform, and information about which games were bundled together.

In [9]:
#export
import os
import ast
import json

import pandas as pd
import numpy as np

from recohut.utils.common_utils import *
from recohut.datasets.bases.common import Dataset

In [14]:
#export
class SteamDataset(Dataset):
    games_url = 'https://github.com/RecoHut-Datasets/steam/raw/v1/steam_games.json.gz'
    users_url = 'https://github.com/RecoHut-Datasets/steam/raw/v1/australian_users_items.json.gz'
    
    def __init__(self, data_dir):
        super().__init__(data_dir)
        
        self._process()
 
    @property
    def raw_file_names(self) -> str:
        return ['steam_games.json', 'australian_users_items.json']

    @property
    def processed_file_names(self) -> str:
        return ['gamesdata.parquet.snappy', 'mergeddata.parquet.snappy', 
                'numgames.parquet.snappy', 'recdata.parquet.snappy']

    def download(self):
        for url in [self.games_url, self.users_url]:
            path = download_url(url, self.raw_dir)
            extract_gz(path, self.raw_dir)
            os.unlink(path)

    def process(self):
        with open(self.raw_paths[0]) as f:
            lines = f.readlines()
        lines_list = []
        for l in lines:
            _x = '[' + l + ']'
            _x =  ast.literal_eval(_x)
            lines_list.extend(_x)
        temp_path = os.path.join(self.processed_dir, 'meta.json')
        with open(temp_path, 'w') as json_file:
            json.dump(lines_list, json_file)
        df = pd.read_json(temp_path)
        df.price = df.price.astype('str')
        df.metascore = df.metascore.astype('str')
        os.unlink(temp_path)

        with open(self.raw_paths[1]) as f:
            lines = f.readlines()
        lines_list = []
        for l in lines:
            _x = '[' + l + ']'
            _x =  ast.literal_eval(_x)
            lines_list.extend(_x)
        temp_path = os.path.join(self.processed_dir, 'data.json')
        with open(temp_path, 'w') as json_file:
            json.dump(lines_list, json_file)
        useritems = pd.read_json(temp_path)
        os.unlink(temp_path)

        useritems['item_id'] = useritems['items'].apply(lambda x: [x [index]['item_id'] for index, _ in enumerate(x)])
        # Add a column with substitute user_id, counter
        useritems['uid'] = np.arange(len(useritems))
        # numgames dataframe for later use
        numgames = useritems[['user_id', 'items_count']]
        # Take relevant columns
        useritems = useritems[['uid', 'item_id']]
        # Explode item_ids into seperate rows
        lst_col = 'item_id'
        useritems = pd.DataFrame({col:np.repeat(useritems[col].values, useritems[lst_col].str.len())
                                    for col in useritems.columns.difference([lst_col])
                                }).assign(**{lst_col:np.concatenate(useritems[lst_col].values)})[useritems.columns.tolist()]
        # Add binary owned column
        useritems['owned'] = np.ones(shape = useritems.shape[0])
        # Change item_id to int
        useritems['item_id'] = useritems['item_id'].astype(int)
        # Rename column to match
        useritems = useritems.rename(columns={'item_id': 'id'})
        # Merge useritems and games data dataframes
        alldata = pd.merge(useritems, df, on = 'id')
        # Drop entries with no title
        datawithnames = alldata.dropna(axis=0, subset=['title'])
        # Get relevant columns for recommendation engine
        recdata = datawithnames[['uid','id','owned']]

        df.to_parquet(self.processed_paths[0], compression='snappy')
        datawithnames.to_parquet(self.processed_paths[1], compression='snappy')
        numgames.to_parquet(self.processed_paths[2], compression='snappy')
        recdata.to_parquet(self.processed_paths[3], compression='snappy')

Example

In [ ]:
ds = SteamDataset(data_dir='/content/data')

Processing...
